In [ ]:
import httpx
import os
cookies = {'session': os.environ['AOC_SESSION']}
res = httpx.get('https://adventofcode.com/2025/day/11/input', cookies=cookies)

In [ ]:
instructions = list(res.iter_lines())

## --- Day 11: Reactor ---

You hear some loud beeping coming from a hatch in the floor of the factory, so you decide to check it out. Inside, you find several large electrical conduits and a ladder.

Climbing down the ladder, you discover the source of the beeping: a large, toroidal reactor which powers the factory above. Some Elves here are hurriedly running between the reactor and a nearby server rack, apparently trying to fix something.

One of the Elves notices you and rushes over. "It's a good thing you're here! We just installed a new *server rack*, but we aren't having any luck getting the reactor to communicate with it!" You glance around the room and see a tangle of cables and devices running from the server rack to the reactor. She rushes off, returning a moment later with a list of the devices and their outputs (your puzzle input).

For example:

```
aaa: you hhh
you: bbb ccc
bbb: ddd eee
ccc: ddd eee fff
ddd: ggg
eee: out
fff: out
ggg: out
hhh: ccc fff iii
iii: out
```

Each line gives the name of a device followed by a list of the devices to which its outputs are attached. So, `bbb: ddd eee` means that device `bbb` has two outputs, one leading to device `ddd` and the other leading to device `eee`.

The Elves are pretty sure that the issue isn't due to any specific device, but rather that the issue is triggered by data following some specific *path* through the devices. Data only ever flows from a device through its outputs; it can't flow backwards.

After dividing up the work, the Elves would like you to focus on the devices starting with the one next to you (an Elf hastily attaches a label which just says *`you`*) and ending with the main output to the reactor (which is the device with the label *`out`*).

To help the Elves figure out which path is causing the issue, they need you to find *every* path from `you` to `out`.

In this example, these are all of the paths from `you` to `out`:

- Data could take the connection from `you` to `bbb`, then from `bbb` to `ddd`, then from `ddd` to `ggg`, then from `ggg` to `out`.
- Data could take the connection to `bbb`, then to `eee`, then to `out`.
- Data could go to `ccc`, then `ddd`, then `ggg`, then `out`.
- Data could go to `ccc`, then `eee`, then `out`.
- Data could go to `ccc`, then `fff`, then `out`.

In total, there are **5** different paths leading from `you` to `out`.

**How many different paths lead from `you` to `out`?**

In [ ]:
sample_input = '''aaa: you hhh
you: bbb ccc
bbb: ddd eee
ccc: ddd eee fff
ddd: ggg
eee: out
fff: out
ggg: out
hhh: ccc fff iii
iii: out'''.splitlines()


### Part 1

In [ ]:
sample_input

['aaa: you hhh',
 'you: bbb ccc',
 'bbb: ddd eee',
 'ccc: ddd eee fff',
 'ddd: ggg',
 'eee: out',
 'fff: out',
 'ggg: out',
 'hhh: ccc fff iii',
 'iii: out']

I need to determine how many paths there are that go from "you" to "out". I am thinking this feels like a graph problem so I'm thinking networkx makes the most sense, but I'm open to other ideas if you think there would be something better

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

You're right that this is a graph problem! NetworkX would work, and it has `nx.all_simple_paths(G, source, target)` that could enumerate paths.

However, a couple of things to consider:

1. **If the graph is a DAG** (directed acyclic graph, which this looks like), there's a very efficient approach: **dynamic programming / memoization**. The number of paths from node X to "out" equals the sum of paths from each of X's children to "out". This is O(nodes + edges).

2. **If you use `all_simple_paths`**, it actually generates each path, which could be very slow if there are millions/billions of paths (common in AoC problems designed to blow up naive solutions).

For Part 1, either might work fine. But if Part 2 scales up, the DP approach is safer.

Would you like to:
- **A)** Try networkx first and see if it's fast enough?
- **B)** Go straight to the recursive/memoized counting approach?

Either way, the first step is parsing the input into a graph structure. Want to start there?

Can you describe the dynamic programming approach further?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠

Sure! The key insight is this:

**The number of paths from any node X to "out" equals the sum of paths from each of X's children to "out".**

So if node X points to nodes A, B, and C, then:
```
paths(X) = paths(A) + paths(B) + paths(C)
```

And our base case is simple: `paths("out") = 1` (there's exactly one way to get from "out" to "out" — you're already there!).

By using memoization (caching results), we only compute each node's count once, even if multiple parents point to it.

For your sample graph, working backwards:
- `paths(ggg) = 1` (points to out)
- `paths(eee) = 1` (points to out)
- `paths(ddd) = paths(ggg) = 1`
- `paths(bbb) = paths(ddd) + paths(eee) = 1 + 1 = 2`
- ...and so on up to "you"

Does this make sense so far? Would you like to try implementing it yourself, or would you like me to give you more of a starting point?

I'm still not quite getting it 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

No problem! Let me try a different angle using your sample data.

Think of it like this: imagine you're standing at a node and asking "how many ways can I reach 'out' from here?"

Let's look at the simplest nodes first — the ones that point directly to "out":
- `eee → out` — there's **1 way** to reach out from eee
- `fff → out` — there's **1 way** to reach out from fff  
- `ggg → out` — there's **1 way** to reach out from ggg
- `iii → out` — there's **1 way** to reach out from iii

Now look at `ddd`. It points to `ggg`. So to get from `ddd` to `out`, you *must* go through `ggg`. Since there's 1 way from `ggg` to `out`, there's also **1 way** from `ddd` to `out`.

Now look at `bbb`. It points to *both* `ddd` and `eee`. You could go either way! So the total paths from `bbb` = paths from `ddd` + paths from `eee` = 1 + 1 = **2 ways**.

Does that help clarify how we "add up" the paths from children? Would you like to try working out what `ccc`'s count would be?

ccc = ddd + eee + fff 

ccc = ggg + out + out 

ccc = out + out + out 

ccc = 3

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Exactly right! `ccc` has 3 paths to reach "out". You've got it!

Your notation is a nice way to visualize it — you traced each path all the way down to "out" and counted them up.

Now here's the power of memoization: once we know `ccc = 3`, we store that. So when we calculate `hhh` (which also points to `ccc`), we don't have to re-trace all those paths — we just look up the cached value of 3.

Want to try calculating `you` now? Remember `you` points to `bbb` and `ccc`, and we've already figured out both of those!

In [ ]:
sample_input

['aaa: you hhh',
 'you: bbb ccc',
 'bbb: ddd eee',
 'ccc: ddd eee fff',
 'ddd: ggg',
 'eee: out',
 'fff: out',
 'ggg: out',
 'hhh: ccc fff iii',
 'iii: out']

In [ ]:
tmp = sample_input[0]

In [ ]:
device_lookup = {}
for instruction in sample_input: 
    device, connected_devices = instruction.split(':')
    connected_devices = connected_devices.strip().split(' ')
    device_lookup[device] = connected_devices

In [ ]:
def connection_lookup(device_name, stop_device, lookup_table): 
    counter = 0 
    if device_name == stop_device: 
        return 1 
    connected_devices = lookup_table[device_name]
    for connected_device in connected_devices:
        counter+=connection_lookup(connected_device, stop_device, lookup_table)
    return counter

In [ ]:
connection_lookup('you', 'out', device_lookup)

5

In [ ]:
device_lookup = {}
for instruction in instructions: 
    device, connected_devices = instruction.split(':')
    connected_devices = connected_devices.strip().split(' ')
    device_lookup[device] = connected_devices

In [ ]:
connection_lookup('you', 'out', device_lookup)


714

### Part 2

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=None)
def connection_lookup(device_name, contains_fft, contains_dac): 
    counter = 0 
    if device_name == stop_device:
        return 1 if contains_fft==1 and contains_dac==1 else 0
    if device_name == 'fft': 
        contains_fft = 1
    if device_name == 'dac':
        contains_dac = 1
    connected_devices = lookup_table[device_name]
    for connected_device in connected_devices:
        counter+=connection_lookup(connected_device, contains_fft, contains_dac)
    return counter

In [ ]:
sample_input2 = """svr: aaa bbb
aaa: fft
fft: ccc
bbb: tty
tty: ccc
ccc: ddd eee
ddd: hub
hub: fff
eee: dac
dac: fff
fff: ggg hhh
ggg: out
hhh: out""".splitlines()

In [ ]:
device_lookup = {}
for instruction in sample_input2: 
    device, connected_devices = instruction.split(':')
    connected_devices = connected_devices.strip().split(' ')
    device_lookup[device] = connected_devices

In [ ]:
stop_device, lookup_table = 'out', device_lookup
connection_lookup('svr', 0, 0)

2

In [ ]:
device_lookup = {}
for instruction in instructions: 
    device, connected_devices = instruction.split(':')
    connected_devices = connected_devices.strip().split(' ')
    device_lookup[device] = connected_devices
stop_device, lookup_table = 'out', device_lookup
connection_lookup('svr', 0, 0)


This is taking too long to run.  Can I add an lru_cache to this?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes, `lru_cache` can definitely help here! But there's one catch — `lru_cache` requires all arguments to be **hashable**, and your `lookup_table` dict isn't hashable.

The simplest fix: move `lookup_table` and `stop_device` outside the function (since they don't change between calls), so they don't need to be parameters. Then you can cache based on just `(device_name, contains_fft, contains_dac)`.

Something like:

```python
from functools import lru_cache

@lru_cache(maxsize=None)
def connection_lookup(device_name, contains_fft, contains_dac):
    # use device_lookup and 'out' directly from outer scope
    ...
```

Want to try adapting your function this way?